[View in Colaboratory](https://colab.research.google.com/github/sabumjung/DL-with-TensorFlow/blob/DL-with-TensorFlow/3%EC%9E%A5_3_5%EC%B8%B5_%EC%8B%A0%EA%B2%BD%EB%A7%9D(Relu)%EA%B3%BC_Dropout%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_MNIST_%EB%B6%84%EB%A5%98.ipynb)

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import math

In [0]:
logs_path = 'log_simple_stats_5_lyers_dropout'
batch_size = 100
learning_rate = 0.5
training_epochs = 10

In [5]:
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

X = tf.placeholder(tf.float32, [None, 784])
Y_ = tf.placeholder(tf.float32, [None, 10])
lr = tf.placeholder(tf.float32)
dropout_ratio = tf.placeholder(tf.float32)

L = 200
M = 100
N = 60
O = 30

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.ones([N])/10)
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.ones([O])/10)
W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

In [0]:
# 각 층에 드롭아웃 처리한 모델
XX = tf.reshape(X, [-1, 28*28])

Y1 = tf.nn.relu(tf.matmul(XX, W1) + B1)
Y1d = tf.nn.dropout(Y1, dropout_ratio)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, dropout_ratio)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, dropout_ratio)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, dropout_ratio)

Ylogits = tf.matmul(Y4d, W5) + B5
Y = tf.nn.softmax(Ylogits)

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, \
                                                        labels=Y_)

cross_entropy = tf.reduce_mean(cross_entropy)*100

correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge_all()

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [15]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  writer = tf.summary.FileWriter(logs_path, \
                                 graph=tf.get_default_graph())

  for epoch in range(training_epochs):
    batch_count = int(mnist.train.num_examples/batch_size)
    for i in range(batch_count):
      batch_x, batch_y = mnist.train.next_batch(batch_size)
      max_learning_rate = 0.003
      min_learning_rate = 0.0001
      decay_speed = 2000
      learning_rate = min_learning_rate + (max_learning_rate - \
                      min_learning_rate) * math.exp(-i/decay_speed)
      _, summary = sess.run([train_step, summary_op], {X: batch_x, \
                                                       Y_:batch_y, \
                                                       dropout_ratio: 0.75, \
                                                       lr: learning_rate})
         
      writer.add_summary(summary, epoch * batch_count + i)
    print ("Epoch: ", epoch)
    print ("Accuracy: ", accuracy.eval\
          (feed_dict={X: mnist.test.images, \
                      Y_: mnist.test.labels, dropout_ratio: 0.75}))
    print ("done")

('Epoch: ', 0)
('Accuracy: ', 0.9213)
done
('Epoch: ', 1)
('Accuracy: ', 0.945)
done
('Epoch: ', 2)
('Accuracy: ', 0.9524)
done
('Epoch: ', 3)
('Accuracy: ', 0.955)
done
('Epoch: ', 4)
('Accuracy: ', 0.96)
done
('Epoch: ', 5)
('Accuracy: ', 0.9634)
done
('Epoch: ', 6)
('Accuracy: ', 0.9644)
done
('Epoch: ', 7)
('Accuracy: ', 0.9649)
done
('Epoch: ', 8)
('Accuracy: ', 0.9657)
done
('Epoch: ', 9)
('Accuracy: ', 0.9676)
done
